In [75]:
import subprocess
process = subprocess.Popen("ls",
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE,
                     text = True,
                     shell = True
                     )
std_out, std_err = process.communicate()
std_out.strip(), std_err

('\x1b\x1bcrisprAll.sh\x1b[m\x1b[m\nfastscript.ipynb\n\x1b\x1bfastscript.py\x1b[m\x1b[m\n\x1b\x1bmincedAll.sh\x1b[m\x1b[m\nparse_minced_res.py\nrun_cctyper_new.py\n\x1b\x1brun_minced.py\x1b[m\x1b[m\n\x1b\x1bunzip_MAGs.py\x1b[m\x1b[m\n\x1b\x1bunzip_MAGs_parallel.sh\x1b[m\x1b[m',
 '')

In [66]:
import time
from datetime import datetime
import pathlib
import os
import sys

input_dir="/Users/isaccocenacchi/Desktop/Tirocinio/MAGs"
out="new"
# out=None
inplace=False
if not os.path.exists(input_dir):
    print("The input directory does not exist", file=sys.stderr)
    sys.exit(1)

if inplace:
    if out==None:
        # Create output directory with unique name near the input directory
        output_dir = f"{os.path.abspath(input_dir)}_unzip_{time.strftime('%Y%m%d%H%M%S')}"
    else:
        # Create output directory with name specified by the user near the input directory
        output_dir = os.path.join(os.path.abspath(input_dir).removesuffix(os.path.basename(input_dir)), os.path.basename(out))
else:
    if out==None:
        # Create output directory with unique name in the out directory of the current working directory
        output_dir = os.path.join(os.getcwd(), "out", f"{os.path.basename(input_dir)}_unzip_{time.strftime('%Y%m%d%H%M%S')}")
    else:
        # Create output directory with name specified by the user in the out directory of the current working directory
        output_dir = os.path.join(os.getcwd(), "out", os.path.basename(out))

print("Input: " + input_dir)
print("Output: " + output_dir)
os.path.abspath(input_dir).split('/')[:-1]
os.path.abspath(input_dir).removesuffix(os.path.basename(input_dir))
# os.path.basename(input_dir)


Input: /Users/isaccocenacchi/Desktop/Tirocinio/MAGs
Output: /Users/isaccocenacchi/Desktop/Tirocinio/scripts/out/new


'/Users/isaccocenacchi/Desktop/Tirocinio/'

In [199]:
from glob import glob
import os
import bz2
import argparse
import time
import subprocess
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import multiprocessing as mp
import itertools


# run_minced.py
# Version 0.1
# 18/04/2024
# by isacchetto

input_directory = '/Users/isaccocenacchi/Desktop/Tirocinio/MAGs_short'
output_dir = "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S"))
measures = []

# mags = glob(os.path.join(input_directory, '**/*.bz2'), recursive=True)

# mags = []
# for root, _, files in os.walk(input_directory):
#         for filename in files:
#             if filename.endswith('.bz2'):
#                   mags.append(os.path.join(root, filename))
# print(len(mags))

mags = [os.path.join(dirpath,filename) 
             for dirpath, _, filenames in os.walk(input_directory) 
             for filename in filenames 
             if filename.endswith('.bz2')
            ]
print("Mags: ", len(mags))
num_cpus = mp.cpu_count() - 1
print("CPUs: ", num_cpus)


Mags:  586
CPUs:  7


In [ ]:
def decompress1(mags, output_dir):
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        with open(unzipped_mag, 'wb') as new_file, bz2.BZ2File(mag, 'rb') as file:
            for data in iter(lambda : file.read(100 * 1024), b''):
                new_file.write(data)
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

def batched(iterable, n):
    it = iter(iterable)
    while (batch := tuple(itertools.islice(it, n))):
        yield batch

with mp.Pool(num_cpus) as pool:
        pool.map(decompress1, batched(mags, num_cpus))

In [ ]:
times = pd.Series(measures)
times.head()

times.plot.line()
plt.xlabel('MAG')
plt.ylabel('Time (s)')
plt.title('Time to unzip MAGs')

plt.show()

In [171]:
def decompress2(mags, output_dir):
    measures.clear()
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        with open(unzipped_mag, 'wb') as new_file, open(mag, 'rb') as file:
            decompressor = bz2.BZ2Decompressor()
            for data in iter(lambda : file.read(100 * 1024), b''):
                new_file.write(decompressor.decompress(data))
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

decompress2(mags, "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S")))

Tempo medio: 0.3039279778798421 secondi
Tempo totale: 0.9117839336395264 secondi


In [ ]:
from multiprocessing import Pool
import subprocess

def run_script(input):
    (command,arg_str)=input
    print("Starting command :{} with argument {}".format(command, arg_str))
    result = subprocess.call(command+" "+arg_str, shell=True) 
    print("Completed command :{} with argument {}".format(command, arg_str))
    return result

if __name__ == '__main__':
    with Pool(5) as p: # choose appropriate level of parallelism
        # choose appropriate command and argument, can be fetched from sys.argv if needed
        exit_codes = p.map(run_script, [('echo','hello1'), ('echo','hello2')])
        print("Exit codes : {}".format(exit_codes))

In [ ]:
def decompress4(mags, output_dir):
    measures.clear()
    for mag in mags:
        start_time = time.time()
        unzipped_mag=os.path.join(output_dir, os.path.relpath(mag[:-4], input_directory))
        os.makedirs(os.path.dirname(unzipped_mag), exist_ok=True)
        sts=subprocess.Popen(['bunzip2', '-kc', mag, '>', unzipped_mag])
        sts.wait()
        # os.system('bunzip2 -k {}'.format(mag))
        end_time = time.time()
        measures.append(end_time - start_time)
    mean_time = sum(measures) / len(measures)
    print("Tempo medio:", mean_time, "secondi")
    print("Tempo totale:", sum(measures), "secondi")

decompress4(mags, "{}_crispr_{}".format(input_directory, time.strftime("%Y%m%d%H%M%S")))
